In [ ]:
import pandas as pd
import random
from functools import reduce

#Filter out the invalid entries and make one-hot vectors for gender:

print("lendo arquivo...")
produtos = pd.read_json('data',lines=True)
print("arquivo lido!")

In [ ]:
produtos = produtos[['productId','gender']].dropna(subset=['productId'])
produtos_dummies = pd.get_dummies(produtos['gender'])
produtos = pd.concat([produtos,produtos_dummies], axis=1)[['productId','F','M']]

In [ ]:
generate_prob_fun = lambda df: (df['M'] + 1)/(df['M'] + df['F'] + 2)

produtos = produtos.groupby('productId').sum().assign(probMale = generate_prob_fun)

In [ ]:
notFoundProd = pd.DataFrame({'productId': ['notanumber'],
              'F': [0.0],
              'M': [0.0],
              'probMale': [.5]}).set_index('productId')
produtos = pd.concat([produtos, notFoundProd])

In [ ]:
target  = pd.read_json("target", lines=True)[['uid', 'productId']]
target.productId = target.productId.fillna('notanumber')

In [ ]:

reduce_prob_fun = lambda x,y : (x*y) / ( (x*y) + ((1-x)*(1-y)))
def agg_two_probabilities(group):
    #Products that doesn't have the probability can be has a 0.5 of probability 
    return reduce(reduce_prob_fun, group.probMale)

uidProbs = target.set_index('productId')\
                 .join(produtos[['probMale']])\
                 .reset_index(drop=True)\
                 .groupby('uid').apply(agg_two_probabilities)\
                 .to_frame()

In [ ]:
def make_response(df):
    df.loc[uidProbs[0] < 0.5, 'gender'] = 'F'
    df.gender = df.gender.fillna("M")
    return df.drop(0, axis = 1)\
             .reset_index()
             
        
#Actually this isn't necessary, i just do it to show how pipe works
response = uidProbs.rename(columns={'uid':'userId'})\
                   .pipe(make_response)
response.to_csv('attempt2', index=False)